In [1]:
import tensorflow as tf
import pathlib
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
t1 = tf.constant([
[1,2,3],
[4,5,6],
[7,8,9],
],dtype=tf.float32)
ds1 = tf.data.Dataset.from_tensors(t1)
ds2 = tf.data.Dataset.from_tensor_slices(t1)

In [3]:
for element in ds1:
    print(element)

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]], shape=(3, 3), dtype=float32)


In [4]:
for element in ds2:
    print(element)

tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32)
tf.Tensor([4. 5. 6.], shape=(3,), dtype=float32)
tf.Tensor([7. 8. 9.], shape=(3,), dtype=float32)


In [5]:
train, test = tf.keras.datasets.fashion_mnist.load_data()
images, labels = train
images = images/255
type(images),type(labels)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


(numpy.ndarray, numpy.ndarray)

In [6]:
dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset


<_TensorSliceDataset element_spec=(TensorSpec(shape=(28, 28), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.uint8, name=None))>

In [7]:
directory_url ="https://storage.googleapis.com/download.tensorflow.org/data/illiad/"
file_names = ['cowper.txt', 'derby.txt', 'butler.txt']
file_paths = [
tf.keras.utils.get_file(file_name, directory_url + file_name)
for file_name in file_names
]

815980/815980 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
809730/809730 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
807992/807992 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
text_line_dataset = tf.data.TextLineDataset(file_paths)

In [9]:
for line in text_line_dataset.take(5):
    print(line.numpy())

b"\xef\xbb\xbfAchilles sing, O Goddess! Peleus' son;"
b'His wrath pernicious, who ten thousand woes'
b"Caused to Achaia's host, sent many a soul"
b'Illustrious into Ades premature,'
b'And Heroes gave (so stood the will of Jove)'


In [10]:
data_url ="https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz"
dataset_dir = tf.keras.utils.get_file(
    origin=data_url,
    untar=True,
    cache_dir= "stack_overflow",
    cache_subdir = ""
)
print(dataset_dir)
dataset_dir = pathlib.Path(dataset_dir)
train_dir = dataset_dir/'train'
train_dir

6053168/6053168 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
/tmp/.keras/stack_overflow_16k.tar.gz


PosixPath('/tmp/.keras/stack_overflow_16k.tar.gz/train')

In [11]:
batch_size = 32
seed = 42
raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed
)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


In [12]:
titanic_file = tf.keras.utils.get_file(
    "titanic.csv",
    "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
)

df = pd.read_csv(titanic_file)


60302/60302 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [13]:
import pandas as pd
import tensorflow as tf
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

numeric_cols = ['Age', 'Fare', 'SibSp', 'Parch']
df[numeric_cols] = df[numeric_cols].fillna(0)

categorical_cols = ['Sex', 'Cabin', 'Embarked', 'Pclass']
df[categorical_cols] = df[categorical_cols].fillna('')
for col in categorical_cols:
    df[col] = df[col].astype('category').cat.codes
df = df.drop(columns=['Name', 'Ticket', 'PassengerId'])
titanic_dataset = tf.data.Dataset.from_tensor_slices(dict(df))
for example in titanic_dataset.take(1):
    print(example)


{'Survived': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'Pclass': <tf.Tensor: shape=(), dtype=int8, numpy=2>, 'Sex': <tf.Tensor: shape=(), dtype=int8, numpy=1>, 'Age': <tf.Tensor: shape=(), dtype=float64, numpy=22.0>, 'SibSp': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'Parch': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'Fare': <tf.Tensor: shape=(), dtype=float64, numpy=7.25>, 'Cabin': <tf.Tensor: shape=(), dtype=int16, numpy=0>, 'Embarked': <tf.Tensor: shape=(), dtype=int8, numpy=3>}


In [14]:
for feature_batch in titanic_dataset.take(1):
    for key,value in feature_batch.items():
        print(" {!r:20s}: {}".format(key,value))

 'Survived'          : 0
 'Pclass'            : 2
 'Sex'               : 1
 'Age'               : 22.0
 'SibSp'             : 1
 'Parch'             : 0
 'Fare'              : 7.25
 'Cabin'             : 0
 'Embarked'          : 3


In [15]:
import tensorflow as tf
import pandas as pd
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
local_file = tf.keras.utils.get_file("titanic.csv", url)
titanic_batches = tf.data.experimental.make_csv_dataset(
    local_file,
    batch_size=4,
    label_name="Survived",
    select_columns=['Pclass', 'Fare', 'Survived'],
    na_value="",
    num_epochs=1,
    shuffle=True
)

for feature_batch, label_batch in titanic_batches.take(1):
    print(f"Survived: {label_batch}")
    for key, value in feature_batch.items():
        print(f"{key:20s}: {value}")

Survived: [1 0 0 1]
Pclass              : [2 3 3 1]
Fare                : [29.      7.8958  7.75   91.0792]


In [16]:
import pandas as pd
import tensorflow as tf
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
local_file = tf.keras.utils.get_file("titanic.csv", url)
df = pd.read_csv(local_file)
df[['Age', 'Fare', 'SibSp', 'Parch', 'Pclass', 'Survived']] = df[['Age', 'Fare', 'SibSp', 'Parch', 'Pclass', 'Survived']].fillna(0)
df[['Sex', 'Cabin', 'Embarked', 'Name', 'Ticket']] = df[['Sex', 'Cabin', 'Embarked', 'Name', 'Ticket']].fillna('')
for col in ['Sex', 'Embarked', 'Cabin']:
    df[col] = df[col].astype('category').cat.codes

df = df.drop(columns=['Name', 'Ticket', 'PassengerId'])
titanic_dataset = tf.data.Dataset.from_tensor_slices((dict(df.drop('Survived', axis=1)), df['Survived']))
titanic_dataset = titanic_dataset.batch(4)
for features, labels in titanic_dataset.take(1):
    print("Features:", features)
    print("Labels:", labels)


Features: {'Pclass': <tf.Tensor: shape=(4,), dtype=int64, numpy=array([3, 1, 3, 1])>, 'Sex': <tf.Tensor: shape=(4,), dtype=int8, numpy=array([1, 0, 0, 0], dtype=int8)>, 'Age': <tf.Tensor: shape=(4,), dtype=float64, numpy=array([22., 38., 26., 35.])>, 'SibSp': <tf.Tensor: shape=(4,), dtype=int64, numpy=array([1, 1, 0, 1])>, 'Parch': <tf.Tensor: shape=(4,), dtype=int64, numpy=array([0, 0, 0, 0])>, 'Fare': <tf.Tensor: shape=(4,), dtype=float64, numpy=array([ 7.25  , 71.2833,  7.925 , 53.1   ])>, 'Cabin': <tf.Tensor: shape=(4,), dtype=int16, numpy=array([ 0, 82,  0, 56], dtype=int16)>, 'Embarked': <tf.Tensor: shape=(4,), dtype=int8, numpy=array([3, 1, 3, 3], dtype=int8)>}
Labels: tf.Tensor([0 1 1 1], shape=(4,), dtype=int64)


In [17]:
flowers_root = tf.keras.utils.get_file(
'flower_photos',
"https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz",
untar=True
)
flowers_root = pathlib.Path(flowers_root)

228813984/228813984 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [18]:
for item in flowers_root.glob("*"):
    print(item)

/root/.keras/datasets/flower_photos/flower_photos


In [19]:
file_path_ds = tf.data.Dataset.list_files(str(flowers_root/'*/*'))

In [20]:
def process_path(file_path):
    label = tf.strings.split(file_path,os.sep)[-2]
    return tf.io.read_file(file_path),label

In [21]:
labeled_ds = file_path_ds.map(process_path)

In [23]:
for image_raw, label in labeled_ds.take(1):
    print(image_raw,label,sep="\n")

tf.Tensor(b"All images in this archive are licensed under the Creative Commons By-Attribution License, available at:\nhttps://creativecommons.org/licenses/by/2.0/\nThe photographers are listed below, thanks to all of them for making their work available, and please be sure to credit them for any use as per the license.\n\ndaisy/7568630428_8cf0fc16ff_n.jpg CC-BY by A Guy Taking Pictures - https://www.flickr.com/photos/80901381@N04/7568630428/\ndaisy/7410356270_9dff4d0e2e_n.jpg CC-BY by martinak15 - https://www.flickr.com/photos/martinaphotography/7410356270/\ndaisy/4286053334_a75541f20b_m.jpg CC-BY by jenny downing - https://www.flickr.com/photos/jenny-pics/4286053334/\ndaisy/10770585085_4742b9dac3_n.jpg CC-BY by Thangaraj Kumaravel - https://www.flickr.com/photos/kumaravel/10770585085/\ndaisy/8759177308_951790e00d_m.jpg CC-BY by Marina del Castell - https://www.flickr.com/photos/marinadelcastell/8759177308/\ndaisy/4131565290_0585c4dd5a_n.jpg CC-BY by jenny downing - https://www.flickr.

In [24]:
inc_dataset = tf.data.Dataset.range(100)
dec_dataset = tf.data.Dataset.range(0, -100, -1)
dataset = tf.data.Dataset.zip((inc_dataset, dec_dataset))
batched_dataset = dataset.batch(4)
for batch in batched_dataset.take(4):
    print([arr.numpy() for arr in batch])

[array([0, 1, 2, 3]), array([ 0, -1, -2, -3])]
[array([4, 5, 6, 7]), array([-4, -5, -6, -7])]
[array([ 8,  9, 10, 11]), array([ -8,  -9, -10, -11])]
[array([12, 13, 14, 15]), array([-12, -13, -14, -15])]


In [25]:
batched_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [26]:
dataset.batch(4,drop_remainder = True)

<_BatchDataset element_spec=(TensorSpec(shape=(4,), dtype=tf.int64, name=None), TensorSpec(shape=(4,), dtype=tf.int64, name=None))>

In [27]:
dataset = tf.data.Dataset.range(100)
dataset = dataset.map(lambda x: tf.fill([tf.cast(x, tf.int32)], x))
padded_batch_dataset = dataset.padded_batch(4, padded_shapes=(None,))
for batch in padded_batch_dataset.take(2):
    print(batch.numpy())
    print()

[[0 0 0]
 [1 0 0]
 [2 2 0]
 [3 3 3]]

[[4 4 4 4 0 0 0]
 [5 5 5 5 5 0 0]
 [6 6 6 6 6 6 0]
 [7 7 7 7 7 7 7]]



In [28]:
dataset = tf.data.TextLineDataset(titanic_file)
dataset.shuffle(buffer_size=10)

<_ShuffleDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [29]:
import tensorflow as tf
import pandas as pd

titanic_file = tf.keras.utils.get_file(
    "titanic.csv",
    "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
)

dataset = tf.data.TextLineDataset(titanic_file)
dataset = dataset.shuffle(buffer_size=10)

for line in dataset.take(5):
    print(line.numpy())


b'4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S'
b'8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.075,,S'
b'7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,E46,S'
b'9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,,S'
b'PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked'


In [30]:
file_path_ds = tf.data.Dataset.list_files(str(flowers_root/'*/*'))

In [31]:
def parse_image(filename):
    label = tf.strings.split(filename, os.sep)[-2]
    image = tf.io.read_file(filename)
    image = tf.io.decode_jpeg(image)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [128, 128])
    return image, label

In [32]:
image_ds = file_path_ds.map(parse_image)

In [33]:
def show(image,label):
    plt.imshow(image)
    plt.title(label.numpy().decode("utf-8"))
    plt.axis("off")
    plt.show()

In [38]:
for image,label in image_ds.take(3):
    show(image,label)

FailedPreconditionError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Error in user-defined function passed to MapDataset:66 transformation with iterator: Iterator::Root::Prefetch::FiniteTake::ParallelMapV2: /root/.keras/datasets/flower_photos/flower_photos/sunflowers; Is a directory
	 [[{{node ReadFile}}]] [Op:IteratorGetNext] name: 

In [39]:
import tensorflow as tf
import pathlib
import os
import matplotlib.pyplot as plt

# Dataset location
flowers_root = tf.keras.utils.get_file(
    'flower_photos',
    "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz",
    untar=True
)
flowers_root = pathlib.Path(flowers_root)

# ✅ Only image files (not directories)
file_path_ds = tf.data.Dataset.list_files(str(flowers_root / '*/*'))

# Process image path
def process_path(file_path):
    label = tf.strings.split(file_path, os.sep)[-2]
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [180, 180])
    image = image / 255.0
    return image, label

image_ds = file_path_ds.map(process_path)

# Test
for image, label in image_ds.take(3):
    print(image.shape, label.numpy())


FailedPreconditionError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Error in user-defined function passed to MapDataset:79 transformation with iterator: Iterator::Root::Prefetch::FiniteTake::ParallelMapV2: /root/.keras/datasets/flower_photos/flower_photos/daisy; Is a directory
	 [[{{node ReadFile}}]] [Op:IteratorGetNext] name: 

In [40]:
train, test = tf.keras.datasets.fashion_mnist.load_data()
images, labels = train
images = images/255.0
labels = labels.astype(np.int32)

In [41]:
fmnist_train_ds = tf.data.Dataset.from_tensor_slices((images, labels))
fmnist_train_ds = fmnist_train_ds.shuffle(5000).batch(32)
model = tf.keras.Sequential([
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(10)
])
model.compile(optimizer='adam',
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
metrics=['accuracy'])

In [43]:
model.fit(fmnist_train_ds, epochs=2)

Epoch 1/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8499 - loss: 0.4373
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8535 - loss: 0.4240


In [44]:
loss, accuracy = model.evaluate(fmnist_train_ds)
print("Loss :", loss)
print("Accuracy :", accuracy)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8492 - loss: 0.4283
Loss : 0.42437300086021423
Accuracy : 0.8510500192642212


In [45]:
predict_ds = tf.data.Dataset.from_tensor_slices(images).batch(32)
result = model.predict(predict_ds, steps = 10)
print(result.shape)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
(320, 10)
